In [1]:
!pip install transformers
!pip install datasets
# !pip install -U PyYAML
!pip install "ray[default]"
!pip install wandb
!pip install tensorboardX

     |████████████████████████████████| 2.5MB 4.3MB/s 
     |████████████████████████████████| 901kB 50.5MB/s 
     |████████████████████████████████| 3.3MB 59.4MB/s 
     |████████████████████████████████| 266kB 4.1MB/s 
     |████████████████████████████████| 245kB 34.4MB/s 
     |████████████████████████████████| 122kB 34.1MB/s 
     |████████████████████████████████| 51.6MB 103kB/s 
     |████████████████████████████████| 3.1MB 47.2MB/s 
     |████████████████████████████████| 10.1MB 48.8MB/s 
     |████████████████████████████████| 71kB 12.2MB/s 
     |████████████████████████████████| 81kB 14.0MB/s 
     |████████████████████████████████| 133kB 54.2MB/s 
     |████████████████████████████████| 1.3MB 51.1MB/s 
     |████████████████████████████████| 81kB 13.4MB/s 
     |████████████████████████████████| 204kB 63.4MB/s 
     |████████████████████████████████| 92kB 14.5MB/s 
     |████████████████████████████████| 143kB 59.2MB/s 
     |████████████████████████████████| 296kB 52.9MB/

In [3]:
import ray
ray.__version__

'1.4.1'

In [4]:
import transformers
transformers.__version__

'4.8.2'

In [ ]:
from ray import tune
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# saves our models to artifacts in WandB
import wandb
# %env WANDB_LOG_MODEL=true
# %env WANDB_PROJECT=goodreads_success_predictor

In [ ]:
wandb.login(key = '46cb1981ae15765be5bfb5e7c3257d0315a95a1b')

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from transformers import DistilBertPreTrainedModel, DistilBertModel
from transformers.modeling_outputs import SequenceClassifierOutput
from torch import nn
import torch
from torch.nn import CrossEntropyLoss, MSELoss

class DistilBERTForMultipleSequenceClassification(DistilBertPreTrainedModel):
    def __init__(self, config, num_labels1 = 2, num_labels2 = 4):
        super().__init__(config)
        self.num_labels1 = num_labels1
        self.num_labels2 = num_labels2
        print(self.num_labels1, self.num_labels2)
        self.alpha = .5
        self.config = config

        self.distilbert = DistilBertModel(config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier1 = nn.Linear(config.dim, self.num_labels1)
        self.classifier2 = nn.Linear(config.dim, self.num_labels2)
        self.dropout = nn.Dropout(config.dropout)

        self.init_weights()


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        distilbert_output = self.distilbert(
              input_ids=input_ids,
              attention_mask=attention_mask,
              head_mask=head_mask,
              inputs_embeds=inputs_embeds,
              output_attentions=output_attentions,
              output_hidden_states=output_hidden_states,
              return_dict=return_dict,
          )
        hidden_state = distilbert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.pre_classifier(pooled_output)  # (bs, dim)
        pooled_output = nn.ReLU()(pooled_output)  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits1 = self.classifier1(pooled_output)
        logits2 = self.classifier2(pooled_output)
        logits = torch.cat([logits1, logits2], 1)

        loss = None
        if labels is not None:
            #if self.config.problem_type is None:
            #self.config.problem_type = "single_label_classification"
            
            if self.num_labels1 > 1:
                loss_fct1 = CrossEntropyLoss()
                loss1 = loss_fct1(logits1.view(-1, self.num_labels1), labels[:, 0].view(-1))
            else:
                loss_fct1 = MSELoss()
                loss1 = loss_fct1(logits1.view(-1), labels[:, 0].view(-1))

            if self.num_labels2 > 1:
                loss_fct2 = CrossEntropyLoss()
                loss2 = loss_fct2(logits2.view(-1, self.num_labels2), labels[:, 1].view(-1))
            else:
                loss_fct2 = MSELoss()
                loss2 = loss_fct2(logits2.view(-1), labels[:, 1].view(-1))
            loss = self.alpha*loss1 + (1-self.alpha)*loss2 

        if not return_dict:
            output = (logits,) + outputs[2:] #not sure if this works
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=distilbert_output.hidden_states, #hidden_states,
            attentions=distilbert_output.attentions, #attentions,
        )

In [ ]:
import wandb
import os
os.environ['WANDB_CONSOLE'] = 'off'
run = wandb.init()

model_name = 'DistilBERT_multitask_overlap50_dataset_embeddings'

artifact = None
if (model_name == 'DistilBERT_multitask_sentence_tokenized_dataset_embeddings'):
  artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/model-2giwtwvy:v0', type='model')
elif (model_name == 'DistilBERT_multitask_overlap50_dataset_embeddings'):
  artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/model-nlpbosie:v0', type='model')

artifact_dir = artifact.download()

model = DistilBERTForMultipleSequenceClassification.from_pretrained(artifact_dir, num_labels1 = 2, num_labels2 = 8)
model.cuda()

wandb: Currently logged in as: lucaguarro (use `wandb login --relogin` to force relogin)


wandb: Downloading large artifact model-nlpbosie:v0, 255.48MB. 3 files... Done. 0:0:0


2 8


DistilBERTForMultipleSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30523, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
run.finish()

In [ ]:
import pickle
from datasets import DatasetDict

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.modeling_outputs import SequenceClassifierOutput

class Net(nn.Module):

    def __init__(self, pre_classifier_init, classifier_init, do_rate = 0.1):
        super(Net, self).__init__()

        self.pre_classifier = nn.Linear(768, 768)
        self.classifier = nn.Linear(768, 2)
        self.dropout = nn.Dropout(do_rate)

        self.pre_classifier.weight.data.copy_(pre_classifier_init.weight.data)
        self.classifier.weight.data.copy_(classifier_init.weight.data)

        # print(pre_classifier_init.bias.data)
        self.pre_classifier.bias.data.copy_(pre_classifier_init.bias.data)
        self.classifier.bias.data.copy_(classifier_init.bias.data)

        # DOUBLE CHECK IF BIASES ARE BEING SET AS WELL

    def forward(self, x, labels = None):
        # Max pooling over a (2, 2) window
        x = self.pre_classifier(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        logits = self.classifier(x)

        return logits
        # loss = None
        # if labels is not None:
        #   loss_fct = CrossEntropyLoss()
        #   loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # return SequenceClassifierOutput(
        #     loss = loss,
        #     logits = logits
        # )

net = Net(model.pre_classifier, model.classifier1)
print(net)

Net(
  (pre_classifier): Linear(in_features=768, out_features=768, bias=True)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [ ]:
net

Net(
  (pre_classifier): Linear(in_features=768, out_features=768, bias=True)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
def load_data():
  with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
    avg_pld_outs_hf_ds = pickle.load(input_file)
  avg_pld_outs_hf_ds.set_format(type='pt', columns=['meaned_pooled_output', 'success_label'])
  trainset = avg_pld_outs_hf_ds['train']
  valset = avg_pld_outs_hf_ds['validation']
  return trainset, valset

def load_test_data():
  with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
    avg_pld_outs_hf_ds = pickle.load(input_file)
  avg_pld_outs_hf_ds.set_format(type='pt', columns=['meaned_pooled_output', 'success_label'])
  testset = avg_pld_outs_hf_ds['test']
  return testset

In [ ]:
from ray import tune
# from ray.tune.integration.wandb import wandb_mixin
# '''@wandb_mixin
# run = wandb.init()

def train_nn(config, checkpoint_dir, data_dir=None):
  net = Net(model.pre_classifier, model.classifier1, config['do_rate'])
  net.train()
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if torch.cuda.device_count() > 1:
          net = nn.DataParallel(net)
  print(type(net))
  net.to(device)
  # net.cuda()


  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

  trainset, valset = load_data()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=config["batch_size"], shuffle=True)
  valloader = torch.utils.data.DataLoader(valset, batch_size=config["batch_size"], shuffle=True)

  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):

      inputs = data['meaned_pooled_output']
      labels = data['success_label']

      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      epoch_steps += 1

      if i % 10 == 9:
        print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                        running_loss / epoch_steps))
        running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      net.eval()
      with torch.no_grad():
        for i, data in enumerate(valloader, 0):

          inputs_cpu = data['meaned_pooled_output']
          labels_cpu = data['success_label']

          inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
          # inputs.cuda()
          # labels.cuda()

          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)

          all_predictions = np.append(all_predictions, predicted.to('cpu').numpy())
          all_labels = np.append(all_labels, labels_cpu.numpy())

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          loss = criterion(outputs, labels)
          val_loss += loss.cpu().numpy()
          val_steps += 1

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((net.state_dict(), optimizer.state_dict()), path)

      net.train()

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      # s_acc = accuracy_score(all_labels, all_predictions)
      # wandb.log({"val_loss": val_loss / val_steps, "val_accuracy": correct / total})
      tune.report(loss=(val_loss / val_steps), accuracy=correct / total, f1=s_f1, precision=s_precision, recall=s_recall)
  print("Finished Training")

In [ ]:
from ray import tune

In [ ]:
def test_results(net, device="cpu"):
    testset = load_test_data()

    testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

    all_predictions = np.array([])
    all_labels = np.array([])

    net.eval()
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs_cpu = data['meaned_pooled_output']
            labels_cpu = data['success_label']

            inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_predictions = np.append(all_predictions, predicted.to('cpu').numpy())
            all_labels = np.append(all_labels, labels_cpu.numpy())

    s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
    return {
        'precision': s_precision,
        'recall': s_recall,
        'f1': s_f1
    }

In [ ]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.wandb import WandbLoggerCallback
import torch.optim as optim
from functools import partial

def main(num_samples = 15, max_num_epochs = 10):
  config = {
      "lr": tune.loguniform(1e-4, 1e-1),
      "batch_size": tune.choice([16,32,64,128]),
      "num_epochs": tune.choice([1,2]),#,2,3]),#,2,3,5,10,20]),
      "do_rate": tune.uniform(0.1, 0.5),
      "wandb": {
        "project": "AvgPooledOutputClassifier",
        "api_key": "46cb1981ae15765be5bfb5e7c3257d0315a95a1b"
      }
    }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(train_nn, checkpoint_dir='/tmp/ShallowNNModels'),
    config = config,
    resources_per_trial={'gpu': 1},
    metric = 'loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="AvgPooledOutputClassifier",
        group='raytune_hpsearch',
        api_key="46cb1981ae15765be5bfb5e7c3257d0315a95a1b",
        log_config=True
    )])
  
  best_trial = result.get_best_trial(metric="f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["accuracy"]))
  
  best_trained_model = Net(model.pre_classifier, model.classifier1, best_trial.config['do_rate'])
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      # if gpus_per_trial > 1:
      #     best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)

  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/Models/ShallowNNModels/yungclassifier1.pt"
  torch.save(best_trained_model.state_dict(), path)
  return test_results(best_trained_model, device)

In [ ]:
main(num_samples=15)

2021-07-09 20:51:05,414	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-09 20:51:06,608	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-07-09 20:51:06,610	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2021-07-09 20:51:44,683	WARNING worker.py:1123 -- Warning: The actor ImplicitFunc has size 267908209 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-07-09 20:51:45,097	WARNING util.py:162 -- The `start_trial` operation took 3.925 s, which may be a performance bottleneck.


Trial name,status,loc,batch_size,do_rate,lr,num_epochs
DEFAULT_63709_00000,RUNNING,,16,0.213574,0.00105227,2
DEFAULT_63709_00001,PENDING,,128,0.496502,0.0145453,1
DEFAULT_63709_00002,PENDING,,64,0.388518,0.000108764,1
DEFAULT_63709_00003,PENDING,,64,0.427725,0.00381855,1
DEFAULT_63709_00004,PENDING,,64,0.180402,0.000976601,2
DEFAULT_63709_00005,PENDING,,128,0.410034,0.0031913,1
DEFAULT_63709_00006,PENDING,,32,0.312963,0.0576668,1
DEFAULT_63709_00007,PENDING,,64,0.318123,0.00183297,1
DEFAULT_63709_00008,PENDING,,128,0.387019,0.00245907,2
DEFAULT_63709_00009,PENDING,,16,0.12072,0.0140084,1


Problem at: /usr/local/lib/python3.7/dist-packages/ray/tune/integration/wandb.py 197 run


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 761, in init
    run = wi.init()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 422, in init
    backend.ensure_launched()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/backend/backend.py", line 89, in ensure_launched
    self.record_q = self._multiprocessing.Queue()
  File "/usr/lib/python3.7/multiprocessing/context.py", line 102, in Queue
    return Queue(maxsize, ctx=self.get_context())
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 42, in __init__
    self._rlock = ctx.Lock()
  File "/usr/lib/python3.7/multiprocessing/context.py", line 67, in Lock
    return Lock(ctx=self.get_context())
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 162, in __init__
    SemLock.__init__(self, SEMAPHORE, 1, 1, ctx=ctx)
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 80, in __init__
    register(self.

(pid=499) <class '__main__.Net'>


...

2021-07-09 20:56:06,345	WARNING util.py:162 -- The `process_trial_save` operation took 1.562 s, which may be a performance bottleneck.


Trial name,status,loc,batch_size,do_rate,lr,num_epochs,iter,total time (s),loss,accuracy,f1
DEFAULT_63709_00000,TERMINATED,,16,0.213574,0.00105227,2,10,39.6452,0.566963,0.789655,0.783308
DEFAULT_63709_00001,TERMINATED,,128,0.496502,0.0145453,1,2,3.34853,0.579994,0.772414,0.763374
DEFAULT_63709_00002,TERMINATED,,64,0.388518,0.000108764,1,7,25.0834,0.533658,0.758621,0.747011
DEFAULT_63709_00003,TERMINATED,,64,0.427725,0.00381855,1,1,0.325666,0.546979,0.765517,0.755241
DEFAULT_63709_00004,TERMINATED,,64,0.180402,0.000976601,2,1,0.328283,0.54843,0.762069,0.751138
DEFAULT_63709_00005,TERMINATED,,128,0.410034,0.0031913,1,2,3.45193,0.561514,0.768966,0.759319
DEFAULT_63709_00006,TERMINATED,,32,0.312963,0.0576668,1,1,0.325576,0.618329,0.665517,0.672299
DEFAULT_63709_00007,TERMINATED,,64,0.318123,0.00183297,1,1,0.347872,0.552433,0.762069,0.751138
DEFAULT_63709_00008,TERMINATED,,128,0.387019,0.00245907,2,1,0.343681,0.564933,0.758621,0.747011
DEFAULT_63709_00009,TERMINATED,,16,0.12072,0.0140084,1,1,0.337257,0.570796,0.768966,0.759319


2021-07-09 20:56:06,480	INFO tune.py:549 -- Total run time: 299.88 seconds (296.59 seconds for the tuning loop).


Best trial config: {'lr': 0.0010522703449960447, 'batch_size': 16, 'num_epochs': 2, 'do_rate': 0.2135738268782248, 'wandb': {'project': 'AvgPooledOutputClassifier', 'api_key': '46cb1981ae15765be5bfb5e7c3257d0315a95a1b'}}
Best trial final validation loss: 0.5669626656331515
Best trial final validation accuracy: 0.7896551724137931


{'f1': 0.7018430613891009,
 'precision': 0.7034202893634344,
 'recall': 0.7137931034482758}

In [ ]:
avg_pld_outs_hf_ds.set_format(type='pytorch', columns=['meaned_pooled_output', 'success_label'])
trainloader = torch.utils.data.DataLoader(avg_pld_outs_hf_ds['train'], batch_size=128)
net.train()